In [1]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import torch.nn as nn
from pathlib import Path
import pandas as pd
import numpy as np
import random
import torch

from rdkit import Chem

from library.GCN import ConvolutionLayer, PoolingLayer, GraphData, collate_graph_dataset, Standardizer, Graph
from library.cVAE import GCN_Encoder, GRU_Decoder, cVAE
from torch.utils.data import DataLoader,SubsetRandomSampler

In [2]:
def get_vocab(smiles_list):
    # Collect all unique characters
    charset = set()
    for smi in smiles_list:
        for ch in smi:
            charset.add(ch)

    # Sort for consistency
    charset = sorted(list(charset))

    # Add special tokens
    special_tokens = ['<PAD>', '<END>', '<STR>']
    vocab_list = special_tokens + charset

    return vocab_list, len(vocab_list)

In [3]:
# Load QM9 SMILES
df_qm9 = pd.read_pickle('../data/RDKit/rdkit_only_valid_smiles_qm9.pkl')
smiles_list = df_qm9["SMILES"].to_list()

vocab_list, vocab_size = get_vocab(smiles_list)

# Create token2index mapping and its inverse
token2idx = {tok: idx for idx, tok in enumerate(vocab_list)}
idx2token = {idx: tok for tok, idx in token2idx.items()}

print("Vocabulary size:", vocab_size)
print("Example tokens:", vocab_list)

Vocabulary size: 24
Example tokens: ['<PAD>', '<END>', '<STR>', '#', '(', ')', '+', '-', '/', '1', '2', '3', '4', '5', '=', '@', 'C', 'F', 'H', 'N', 'O', '[', '\\', ']']


In [4]:
def padding(smiles):
    # find max length
    max_seq_len = max(s.size(0) for s in smiles)

    # pad with a PAD token index
    pad_idx = token2idx['<PAD>']

    padded_smiles = torch.full((len(smiles), max_seq_len), pad_idx, dtype=torch.long)

    for i, seq in enumerate(smiles):
        padded_smiles[i, :seq.size(0)] = seq
    
    return padded_smiles

In [5]:
def smiles_to_idxs(smiles):
    modified_smiles = [] # [batch_size, seq_len]
    for smile in smiles:
        # Add the start and end tokens to the smiles
        char_list = ['<STR>'] + list(smile) + ['<END>']
        # Convert tokens to indices
        vocab_idx_list = torch.as_tensor([token2idx[ch] for ch in char_list])
        modified_smiles.append(vocab_idx_list)
    
    padded_smiles = padding(modified_smiles)
    return padded_smiles

In [6]:
def smiles_level_accuracy(logits, targets, pad_idx=None):
    """
    logits:  (batch, seq_len, vocab_size)
    targets: (batch, seq_len)
    pad_idx: integer for PAD token (optional)
    """
    # Predicted token IDs
    preds = torch.argmax(logits, dim=-1)  # (batch, seq_len)

    # Boolean matrix: correct prediction per position
    correct = (preds == targets)  # (batch, seq_len)

    # Mask out padding positions
    if pad_idx is not None:
        mask = (targets != pad_idx).float()
        correct = correct.float() * mask
        lengths = mask.sum(dim=1)               # seq_len per SMILES (no pads)
    else:
        correct = correct.float()
        lengths = torch.full(
            (targets.size(0),), targets.size(1), device=targets.device
        )

    # Per-SMILES reconstruction accuracy
    per_smiles_acc = correct.sum(dim=1) / lengths   # (batch,)

    # Average over all SMILES in the batch
    avg_acc = per_smiles_acc.mean()

    return avg_acc.item(), per_smiles_acc


In [7]:

def train_model(
    epoch,
    model,
    training_dataloader,
    optimizer,
    loss_fn,
    use_GPU,
    max_atoms,
    node_vec_len,
):
    """
    Custom function which defines how a model will be trained (per epoch), here the mean-squared loss between prediction and actual value is used as evaluation metric. This function will perform backpropagation which updates the weights of the networks based in this evaluation.
    """
    # Create variables to store losses and error
    avg_loss = 0
    avg_validity = 0
    count = 0
    epoch_acc_sum = 0.0
    epoch_smiles_count = 0
    # Switch model to train mode
    model.train()
    # Go over each batch in the dataloader
    for i, dataset in enumerate(training_dataloader):
        # Unpack data
        node_mat = dataset[0][0]
        adj_mat = dataset[0][1]
        gap = dataset[1]
        
        padded_smiles = smiles_to_idxs(dataset[2])
        batch_size, max_seq_len = padded_smiles.size()

        # Reshape inputs
        node_mat = node_mat.reshape(batch_size, max_atoms, node_vec_len)
        adj_mat = adj_mat.reshape(batch_size, max_atoms, max_atoms)

        # Package inputs and outputs; check if GPU is enabled
        if use_GPU:
            model_input = (node_mat.cuda(), adj_mat.cuda(), padded_smiles, gap.cuda())
            model_output = padded_smiles
        else:
            model_input = (node_mat, adj_mat, padded_smiles, gap)
            model_output = padded_smiles

        # Compute output from network
        model_prediction_distribution = model(*model_input) # [batch_size, max_smiles_seq_len, vocab_size]

        # Calculate loss
        loss = loss_fn(model, model_prediction_distribution.permute(0, 2, 1), model_output, batch_size)
        avg_loss += loss.item()

        # returns (batch_avg, per_smiles_vector)
        batch_avg, per_smiles_acc = smiles_level_accuracy(model_prediction_distribution, padded_smiles)
        epoch_acc_sum += per_smiles_acc.sum().item()   # sum of accuracies for this batch
        epoch_smiles_count += per_smiles_acc.size(0)   # number of molecules in batch

        # max_prob_smiles_tokens = torch.argmax(model_prediction_distribution, dim=-1)
        # max_prob_smiles = "".join(idx2token[idx] for idx in max_prob_smiles_tokens[0].numpy() if idx not in (0, 1, 2))
        
        # Check if the constructed smiles is a valid molecule
        # if Chem.MolFromSmiles(max_prob_smiles) is not None:
        #     avg_validity += 1

        # Set zero gradients for all tensors
        optimizer.zero_grad()
        # Do backward prop
        loss.backward()
        # Update optimizer parameters
        optimizer.step()
        # Increase count
        count += 1
    # Calculate avg loss and validity
    avg_loss = avg_loss / count
    # avg_validity = avg_validity / count
    epoch_smiles_accuracy = epoch_acc_sum / epoch_smiles_count
    # Print stats
    print(f"Epoch: [{epoch}]\tTraining Loss: [{avg_loss:.2f}]\tReconstruction accuracy: [{epoch_smiles_accuracy}]")
    # Return loss and validity
    return avg_loss, epoch_smiles_accuracy

In [8]:

def test_model(
    model,
    test_dataloader,
    loss_fn,
    use_GPU,
    max_atoms,
    node_vec_len,
):
    """
    Test the ChemGCN model.
    Parameters
    ----------
    model : ChemGCN
        ChemGCN model object
    test_dataloader : data.DataLoader
        Test DataLoader
    loss_fn : like nn.MSELoss()
        Model loss function
    standardizer : Standardizer
        Standardizer object
    use_GPU: bool
        Whether to use GPU
    max_atoms: int
        Maximum number of atoms in graph
    node_vec_len: int
        Maximum node vector length in graph
    Returns
    -------
    test_loss : float
        Test loss
    test_mae : float
        Test MAE
    """
    # Create variables to store losses and error
    test_loss = 0
    epoch_acc_sum = 0.0
    epoch_smiles_count = 0
    count = 0
    # Switch model to train mode
    model.eval()
    # Go over each batch in the dataloader
    for i, dataset in enumerate(test_dataloader):
        # Unpack data
        node_mat = dataset[0][0]
        adj_mat = dataset[0][1]
        gap = dataset[1]
        padded_smiles = smiles_to_idxs(dataset[2])
        batch_size, _ = padded_smiles.size()
        # Reshape inputs
        node_mat = node_mat.reshape(batch_size, max_atoms, node_vec_len)
        adj_mat = adj_mat.reshape(batch_size, max_atoms, max_atoms)
        # Package inputs and outputs; check if GPU is enabled
        if use_GPU:
            model_input = (node_mat.cuda(), adj_mat.cuda(), padded_smiles, gap.cuda())
            model_output = padded_smiles
        else:
            model_input = (node_mat, adj_mat, padded_smiles, gap)
            model_output = padded_smiles
        # Compute output from network
        model_prediction_distribution = model(*model_input) # [batch_size, max_smiles_seq_len, vocab_size]
        # Calculate loss
        loss = loss_fn(model, model_prediction_distribution.permute(0, 2, 1), model_output, batch_size)
        test_loss += loss.item()
        # returns (batch_avg, per_smiles_vector)
        batch_avg, per_smiles_acc = smiles_level_accuracy(model_prediction_distribution, padded_smiles)
        epoch_acc_sum += per_smiles_acc.sum().item()   # sum of accuracies for this batch
        epoch_smiles_count += per_smiles_acc.size(0)   # number of molecules in batch
        
        # Increase count
        count += 1
    # Calculate avg loss and MAE
    test_loss = test_loss / count
    epoch_smiles_accuracy = epoch_acc_sum / epoch_smiles_count
    # Print stats
    print(f"Training Loss: [{test_loss:.2f}]\tReconstruction accuracy: [{epoch_smiles_accuracy}]")
    # Return loss and validity
    return test_loss, epoch_smiles_accuracy

In [9]:
#### Inputs
n_epochs = 4
batch_size = 1000
train_size = 0.7
learning_rate = 0.01
device = "cpu"

# GCN
max_atoms = 30 # fixed value
node_vec_len = 16 # fixed value
n_hidden = 32
n_conv_layers = 2
n_hidden_layers = 2

# GRU
latent_dim = 16
gru_dim = 16
embedding_dim = 8
n_gru_layers = 2 # stacked GRUs
n_fc_layers = 3 # 3 dense layers after GRU

# cVAE
gcn_hidden_nodes = n_hidden + 1
teacher_forcing_ratio = 0.5

In [10]:
#### Start by creating dataset
main_path = Path.cwd().parents[0]
data_path = main_path / "data" / "RDKit" / "rdkit_only_valid_smiles_qm9.pkl"
dataset = GraphData(dataset_path=data_path, max_atoms=max_atoms, node_vec_len=node_vec_len)

print(dataset.outputs)

#### Split data into training and test sets
# Get train and test sizes
dataset_indices = np.arange(0, len(dataset), 1)
train_size = int(np.round(train_size * len(dataset)))
test_size = len(dataset) - train_size

# Randomly sample train and test indices
train_indices = np.random.choice(dataset_indices, size=train_size, 
                                                            replace=False)
test_indices = np.array(list(set(dataset_indices) - set(train_indices)))

# Create dataoaders
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, 
                          sampler=train_sampler, 
                          collate_fn=collate_graph_dataset)
test_loader = DataLoader(dataset, batch_size=batch_size, 
                         sampler=test_sampler,
                         collate_fn=collate_graph_dataset)

[13.736308097839355, 9.249149322509766, 9.836915969848633, 9.118535041809082, 10.329442024230957, 6.1579365730285645, 12.0437593460083, 9.352553367614746, 8.767508506774902, 9.90494441986084, 6.372906684875488, 7.741639137268066, 11.371638298034668, 9.29813003540039, 9.347110748291016, 10.694074630737305, 10.133520126342773, 6.3783488273620605, 7.572928428649902, 8.302193641662598, 10.911765098571777, 9.118535041809082, 6.492636680603027, 6.963393688201904, 7.5375542640686035, 5.556564807891846, 5.6218719482421875, 4.231370449066162, 8.424644470214844, 8.5906343460083, 9.668205261230469, 7.768850326538086, 7.815110206604004, 8.269539833068848, 6.2450127601623535, 7.7579665184021, 7.858648300170898, 5.975620269775391, 11.175716400146484, 9.330784797668457, 9.328063011169434, 8.65049934387207, 10.171616554260254, 10.027395248413086, 8.90084457397461, 8.615124702453613, 10.715843200683594, 8.93349838256836, 7.031421661376953, 6.879037857055664, 7.034143447875977, 6.653183937072754, 6.8545

In [ ]:
for dataset in train_loader:
    print(*dataset[1])
    print(dir(dataset))
    break

tensor([4.1633]) tensor([6.4001]) tensor([7.9267]) tensor([6.1498]) tensor([7.8723]) tensor([8.5036]) tensor([8.4410]) tensor([6.8763]) tensor([5.0450]) tensor([6.0600]) tensor([7.5430]) tensor([7.2763]) tensor([7.2083]) tensor([5.9865]) tensor([8.8791]) tensor([6.6913]) tensor([7.3824]) tensor([8.7621]) tensor([6.7239]) tensor([7.4369]) tensor([6.3539]) tensor([4.9389]) tensor([7.2165]) tensor([7.0341]) tensor([5.5892]) tensor([7.5158]) tensor([8.3566]) tensor([8.8083]) tensor([5.3470]) tensor([8.3865]) tensor([7.6219]) tensor([5.4450]) tensor([7.0124]) tensor([6.1443]) tensor([8.6913]) tensor([5.5756]) tensor([9.2002]) tensor([7.0777]) tensor([5.9321]) tensor([6.7022]) tensor([8.1498]) tensor([5.2164]) tensor([7.0260]) tensor([7.4178]) tensor([7.0097]) tensor([8.0138]) tensor([5.4695]) tensor([5.0477]) tensor([7.9784]) tensor([7.6491]) tensor([7.6736]) tensor([6.9852]) tensor([6.5443]) tensor([7.5158]) tensor([8.3512]) tensor([4.8763]) tensor([7.4423]) tensor([3.7661]) tensor([8.3865

In [14]:
dir(train_loader)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_index_sampler',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'in_order',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [ ]:
encoder = GCN_Encoder(
    node_vec_len=node_vec_len,
    node_fea_len=n_hidden,
    hidden_fea_len=n_hidden,
    n_conv=n_conv_layers,
    n_hidden=n_hidden_layers,
    n_outputs=1,
    p_dropout=0.1
)

decoder = GRU_Decoder(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    latent_dim=latent_dim,
    hidden_size=gru_dim,
    n_gru_layers=n_gru_layers,
    n_fc_layers=n_fc_layers
).to(device)

model = cVAE(
    encoder=encoder,
    decoder=decoder,
    device=device,
    n_gcn_hidden_dim=gcn_hidden_nodes,
    n_gru_hidden_dim=gru_dim,
    latent_dim=latent_dim,
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
).to(device)

In [27]:
# Standardizer
# smiles = [dataset[i][2] for i in range(len(dataset))]

# standardizer = Standardizer(torch.Tensor(outputs))

In [28]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
def loss_function(model, logits, targets, batch_size, beta=1):
    recon_loss_fn = nn.CrossEntropyLoss(ignore_index=0)
    loss_recon = recon_loss_fn(logits, targets)
    
    kl_loss = -0.5 * torch.sum(1 + model.z_logvar - model.z_mean.pow(2) - model.z_logvar.exp()) / batch_size
    loss = loss_recon + beta * kl_loss

    return loss

In [30]:
use_GPU = False

#### Train the model
train_loss = []
train_accuracy = []
epoch = []
for i in range(n_epochs):
    epoch_loss, epoch_accuarcy = train_model(
        i,
        model,
        train_loader,
        optimizer,
        loss_function,
        use_GPU,
        max_atoms,
        node_vec_len,
    )
    train_loss.append(epoch_loss)
    epoch.append(i)
    train_accuracy.append(epoch_accuarcy)

Epoch: [0]	Training Loss: [2.69]	Reconstruction accuracy: [0.11064978921973642]
Epoch: [1]	Training Loss: [1.94]	Reconstruction accuracy: [0.23356067077755668]
Epoch: [2]	Training Loss: [1.52]	Reconstruction accuracy: [0.35046428435929206]
Epoch: [3]	Training Loss: [1.33]	Reconstruction accuracy: [0.38661921760899376]


In [35]:
test_loss, test_accuracy = test_model(model, test_loader, loss_function, use_GPU, max_atoms, node_vec_len)

Training Loss: [1.21]	Reconstruction accuracy: [0.41005098154447256]


In [37]:
#### Print final results
print(f"Training Loss: {train_loss[-1]:.2f}")
print(f"Training accuracy: {train_accuracy[-1]:.2f}")
print(f"Test Loss: {test_loss:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")

Training Loss: 1.33
Training accuracy: 0.39
Test Loss: 1.21
Test accuracy: 0.41


Need to add:
- a form of accuracy
- CV for hyperparameter tuning